# Implementación preliminar del código

### Descargar paquetes necesarios

In [49]:
#!pip install catboost
#!pip install sweetviz
#!pip install optuna
#!pip install lightgbm
#!pip install scikit-learn
#!pip install xgboost
#!pip install pyreadstat

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/565.3 kB ? eta -:--:--
     ---------------------------------------- 0.0/565.3 kB ? eta -:--:--
     -------------------------------------- 565.3/565.3 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for pyreadstat: filename=pyreadstat-1.2.9-cp38-cp38-win_amd64.whl size=319963 sha256=57481a12f9b06739deaee8ee58afc526a116a36b435b68ce32f16f777c63cea6
  Stored in directory: c:\users\josé carlos\appdata\local\pip\cache\wheels\34\62\02\7d2524ef9e4a75c2ae00c41eebb26434b133adb534db354842
Successfully built pyreadstat


In [50]:
# --- Librerías estándar ---
import os  # Rutas de los archivos

# --- Librerías científicas y de análisis ---
import matplotlib.pyplot as plt # Gráficos
import numpy as np  # Funciones matemáticas
import pandas as pd  # Manejo de dataframes
import sweetviz as sviz # EDA

# --- Modelos de clasificación ---
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.ensemble import (AdaBoostClassifier, BaggingClassifier, 
                              ExtraTreesClassifier, GradientBoostingClassifier, 
                              RandomForestClassifier)
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

# --- Metricas y evaluación --- 
from sklearn.metrics import (
    accuracy_score, precision_score, f1_score, recall_score
)
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.pipeline import Pipeline

# --- Optimizador ---
import optuna

# --- Elaboración propia ---
from ProbadorHipotesis import ProbadorHipotesis
from ModeladorDatos import ModeladorDatos
from ImportadorDatos import ImportadorDatos
from OptimizadorExtraTreesOptuna import OptimizadorExtraTreesOptuna

## Lectura de datos

Note que la limpieza y transformación de los datos se efectuó en el script "main_enaho.py"

In [2]:
ruta_actual = os.getcwd()
ruta_datos = os.path.join(ruta_actual, '../..', 'data/base.csv') 
ruta_normalizada = os.path.normpath(ruta_datos) 
base = pd.read_csv(ruta_normalizada)

In [3]:
base.head()

,ZONA,REGION,V1,V2A,V3,V4,V5,V6,V6A,V7A,...,V19,V21,R4A,TamViv,A4,A5,A6,CondMig,NivInst,A22A
0,Urbana,Central,En fila o contigua,Alquiler o cesión,Block o ladrillo,Lámina de metal o zinc,Sí,"Mosaico, cerámica, terrazo",No,Bueno,...,SÍ,No,No,2.0,Hombre,45 a 59,Separado/Divorciado/Viudo,No migrante,Secundaria completa,Sí
1,Urbana,Central,En fila o contigua,Alquiler o cesión,Block o ladrillo,Lámina de metal o zinc,Sí,"Mosaico, cerámica, terrazo",No,Bueno,...,SÍ,No,No,2.0,Mujer,13 a 17,Soltero(a),Migrante interno,Primaria completa,No
2,Urbana,Central,En fila o contigua,Propiedad,Otro,Lámina de metal o zinc,Sí,"Mosaico, cerámica, terrazo",No,Bueno,...,SÍ,No,No,3.0,Mujer,45 a 59,Casado(a),Migrante externo,Primaria completa,No
3,Urbana,Central,En fila o contigua,Propiedad,Otro,Lámina de metal o zinc,Sí,"Mosaico, cerámica, terrazo",No,Bueno,...,SÍ,No,No,3.0,Hombre,60 a 74,Casado(a),Migrante externo,Primaria completa,Sí
4,Urbana,Central,En fila o contigua,Propiedad,Otro,Lámina de metal o zinc,Sí,"Mosaico, cerámica, terrazo",No,Bueno,...,SÍ,No,No,3.0,Mujer,25 a 34,Soltero(a),No migrante,Superior,Sí


## Se evaluan los modelos

In [10]:
models = {
    "LogisticRegression": LogisticRegression(),
    "DecisionTree": DecisionTreeClassifier(),
    "SVM": SVC(probability=True),
    "RandomForest": RandomForestClassifier(),
    "XGBoost": XGBClassifier(),
    "KNN": KNeighborsClassifier(),
    "NeuralNetwork": MLPClassifier(),
    "LDA": LinearDiscriminantAnalysis(),
    "CatBoost": CatBoostClassifier(),
    "LightGBM": LGBMClassifier(),
    "AdaBoost": AdaBoostClassifier(),
    "GradientBoosting": GradientBoostingClassifier(),
    "NaiveBayes": GaussianNB(),
    "QDA": QuadraticDiscriminantAnalysis(),
    "Bagging": BaggingClassifier(base_estimator=DecisionTreeClassifier()),
    "ExtraTrees": ExtraTreesClassifier()
}
modelador = ModeladorDatos(df = base, var_objetivo = 'V21', modelos = models)

#### evaluacion de los modelos con el modelador

In [11]:
results_df = modelador.evaluar_modelos()

Training LogisticRegression...


C:\Users\José Carlos\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Training DecisionTree...
Training SVM...
Training RandomForest...
Training XGBoost...
Training KNN...
Training NeuralNetwork...


C:\Users\José Carlos\AppData\Roaming\Python\Python38\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Training LDA...
Training CatBoost...
Learning rate set to 0.036085
0:	learn: 0.6656510	total: 157ms	remaining: 2m 37s
1:	learn: 0.6406955	total: 179ms	remaining: 1m 29s
2:	learn: 0.6167951	total: 220ms	remaining: 1m 12s
3:	learn: 0.5962398	total: 248ms	remaining: 1m 1s
4:	learn: 0.5769672	total: 274ms	remaining: 54.6s
5:	learn: 0.5593462	total: 297ms	remaining: 49.2s
6:	learn: 0.5440616	total: 319ms	remaining: 45.2s
7:	learn: 0.5288220	total: 341ms	remaining: 42.3s
8:	learn: 0.5159265	total: 362ms	remaining: 39.9s
9:	learn: 0.5071103	total: 383ms	remaining: 37.9s
10:	learn: 0.4965033	total: 404ms	remaining: 36.3s
11:	learn: 0.4857068	total: 423ms	remaining: 34.9s
12:	learn: 0.4778379	total: 441ms	remaining: 33.5s
13:	learn: 0.4697625	total: 461ms	remaining: 32.4s
14:	learn: 0.4619196	total: 480ms	remaining: 31.5s
15:	learn: 0.4562249	total: 501ms	remaining: 30.8s
16:	learn: 0.4514157	total: 521ms	remaining: 30.1s
17:	learn: 0.4463129	total: 542ms	remaining: 29.6s
18:	learn: 0.4409676	t

C:\Users\José Carlos\AppData\Roaming\Python\Python38\site-packages\sklearn\discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


Training Bagging...


C:\Users\José Carlos\AppData\Roaming\Python\Python38\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Training ExtraTrees...


In [40]:
# Guardar en Excel
ruta_resultado_modelos = os.path.join(ruta_actual, '../..', 'res/resultados_modelos.xlsx')
ruta_resultado_modelos_norm = os.path.normpath(ruta_resultado_modelos)
results_df.drop(columns=["Predictions"]).to_excel(ruta_resultado_modelos_norm, index=False)
print(results_df)

                 Model  Accuracy   ROC AUC      Gini  Precision    Recall  \
0   LogisticRegression  0.839445  0.851219  0.702438   0.666337  0.412377   
1         DecisionTree  0.912042  0.867564  0.735128   0.776711  0.792892   
2                  SVM  0.865337  0.889795  0.779589   0.805151  0.440564   
3         RandomForest  0.943508  0.980008  0.960016   0.957944  0.753676   
4              XGBoost  0.900768  0.943868  0.887736   0.822844  0.648897   
5                  KNN  0.886397  0.913744  0.827487   0.765011  0.632353   
6        NeuralNetwork  0.911670  0.947178  0.894356   0.791746  0.764093   
7                  LDA  0.834490  0.836813  0.673626   0.649495  0.393995   
8             CatBoost  0.895937  0.936683  0.873365   0.836735  0.602941   
9             LightGBM  0.878097  0.915967  0.831935   0.780763  0.552083   
10            AdaBoost  0.837587  0.845573  0.691146   0.667712  0.391544   
11    GradientBoosting  0.846259  0.865013  0.730025   0.714599  0.398897   

Se considera que el modelo de ExtraTrees es el más adecuado dado a su alto score tanto en precisión como en recall, por lo que se va a utilizar para la optimización de parámetros

#### Se crean los conjuntos de prueba y entrenamiento

In [13]:
X_train, X_test, y_train, y_test = train_test_split(base.drop(['V21'], axis = 1),
                                                    base['V21'],
                                                    test_size = 0.3,
                                                    random_state = 2
                                                    )

### Se crea la funcion a optimizar

In [ ]:
optimizador = OptimizadorExtraTreesOptuna

In [29]:
def objective_extratrees(trial):
    """
    Función objetivo para la optimización de hiperparámetros de un modelo ExtraTreesClassifier mediante Optuna.

    Define un conjunto de hiperparámetros que son sugeridos por Optuna, entrena un pipeline de sklearn
    que incluye preprocesamiento y un clasificador ExtraTrees, y evalúa el modelo mediante validación cruzada
    usando la métrica F1.

    Parámetros
    ----------
    trial : optuna.trial.Trial
        Objeto de prueba generado automáticamente por Optuna para sugerir hiperparámetros en cada iteración.

    Retorna
    -------
    score: float
        Puntuación promedio de validación cruzada (F1 macro) para el conjunto de hiperparámetros sugerido.
        En caso de error durante el entrenamiento o evaluación, retorna `-inf`.
    """
    n_estimators = trial.suggest_int("n_estimators", 100, 500)
    max_depth = trial.suggest_int("max_depth", 5, 40)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 50)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 20)
    max_features = trial.suggest_float("max_features", 0.3, 0.99)
    max_leaf_nodes = trial.suggest_int("max_leaf_nodes", 20, 300)

    modelo = Pipeline(steps=[
        ('preprocessor', modelador.procesador),
        ('classifier', ExtraTreesClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            max_features=max_features,
            max_leaf_nodes=max_leaf_nodes,
            bootstrap=bootstrap,
            random_state=42,
            n_jobs=-1
        ))
    ])

    try:
        score = cross_val_score(modelo, modelador.X_train, modelador.y_train, cv=3, scoring='f1_macro', n_jobs=-1).mean()
        return score if score is not None else float('-inf')
    except Exception as e:
        print(f"Error en el ensayo: {e}")
        return float('-inf')


In [30]:
study = optuna.create_study(direction='maximize', sampler = optuna.samplers.RandomSampler(seed = 3))
study.optimize(objective_extratrees, n_trials=75)

[I 2025-06-07 10:30:41,629] A new study created in memory with name: no-name-2d46060a-e72e-4918-a819-a72ce7849567
[I 2025-06-07 10:31:14,980] Trial 0 finished with value: 0.7595501984805785 and parameters: {'n_estimators': 320, 'max_depth': 30, 'min_samples_split': 16, 'min_samples_leaf': 11, 'max_features': 0.9161333984998816, 'max_leaf_nodes': 271, 'bootstrap': False}. Best is trial 0 with value: 0.7595501984805785.
[I 2025-06-07 10:31:25,466] Trial 1 finished with value: 0.7458220424786397 and parameters: {'n_estimators': 120, 'max_depth': 20, 'min_samples_split': 3, 'min_samples_leaf': 10, 'max_features': 0.7479093928541849, 'max_leaf_nodes': 98, 'bootstrap': True}. Best is trial 0 with value: 0.7595501984805785.
[I 2025-06-07 10:31:34,317] Trial 2 finished with value: 0.7620863782622996 and parameters: {'n_estimators': 109, 'max_depth': 25, 'min_samples_split': 14, 'min_samples_leaf': 9, 'max_features': 0.4956323064222099, 'max_leaf_nodes': 214, 'bootstrap': True}. Best is trial 2

In [32]:
best_parameters = study.best_params
best_score = study.best_value
print(f'Best Hyperparameters: {best_parameters}')
print(f'Best Accuracy: {best_score}')

Best Hyperparameters: {'n_estimators': 128, 'max_depth': 34, 'min_samples_split': 21, 'min_samples_leaf': 3, 'max_features': 0.41992092650360646, 'max_leaf_nodes': 261, 'bootstrap': False}
Best Accuracy: 0.7929152091747991


In [33]:
modelo_final = Pipeline(steps=[
    ('preprocessor', modelador.procesador),
    ('classifier', ExtraTreesClassifier(
        n_estimators = best_parameters['n_estimators'],
        max_depth = best_parameters['max_depth'],
        min_samples_split = best_parameters['min_samples_split'],
        min_samples_leaf = best_parameters['min_samples_leaf'],
        max_features = best_parameters['max_features'],
        max_leaf_nodes = best_parameters['max_leaf_nodes'],
        random_state = 42,
        n_jobs = -1
    ))
])


#### Se ajusta al modelo sobre el conjunto de entrenamiento

In [35]:
modelo_final.fit(modelador.X_train, modelador.y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['V18A1']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['ZONA', 'REGION', 'V1',
                                                   'V2A', 'V3', 'V4', 'V5',
                                                   'V6', 'V6A', 'V7A', 'V7B',
                                                   'V7C', 'EFI', 'V8', 'V11',
                                                   'V12', 'V13A', 'V14A1',
                                                   'V15', 'V16', 'V17A',
                                                   'V17B1', 'V17B2', 'V17B3',
                                                   'V17B4', 'V17B5', 'V18C',
                                                   'V18D', 'V18E', 'V18F', ...])],
                                   verbose_feature_names_out=False)),
                ('classifier',
                 ExtraTreesClassifier(max_depth=34,
                                      max_features=0.41992092650360646,
                                      max_leaf_nodes=261, min_samples_leaf=3,
                                      min_samples_split=21, n_estimators=128,
                                      n_jobs=-1, random_state=42))])

#### Se calculan predicciones con distintas probabilidades y se generan los resultados

In [45]:
results_model = []
y_pred_proba = modelo_final.predict_proba(modelador.X_test)[:, 1]

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

for i in range(0, 11):
    umbral = i * 0.1
    y_pred_class = y_pred_proba >= umbral

    accuracy = accuracy_score(modelador.y_test, y_pred_class)
    precision = precision_score(modelador.y_test, y_pred_class, zero_division=0)
    recall = recall_score(modelador.y_test, y_pred_class, zero_division=0)
    f1 = f1_score(modelador.y_test, y_pred_class, zero_division=0)

    results_model.append((accuracy, precision, recall, f1))


In [46]:
results_model = pd.DataFrame(results_model, columns=['Accuracy', 'Precisión', 'Recall', 'F1'])

ruta_resultado_modelo = os.path.join(ruta_normalizada, '../..', 'res/Resultado_modelo_optuna.xlsx')
results_model.to_excel(ruta_resultado_modelo, index=False)

results_model

,Accuracy,Precisión,Recall,F1
0,0.202180,0.202180,1.000000,0.336356
1,0.691526,0.388456,0.915441,0.545455
2,0.832879,0.558689,0.825368,0.666337
3,0.875124,0.673719,0.741422,0.705951
4,0.888503,0.765217,0.647059,0.701195
5,0.888627,0.832879,0.561887,0.671057
6,0.880203,0.886179,0.467525,0.612114
7,0.862240,0.931894,0.343750,0.502238
8,0.834861,0.977636,0.187500,0.314653
9,0.799802,1.000000,0.009804,0.019417


A partir de estos resultados, se observa que el desempeño global del modelo (según la métrica F1) es óptimo cuando se escoge un umbral de probabilidad alrededor de 30%, lo que representa el mejor balance entre precisión y sensibilidad para predecir la tenencia de bono de vivienda.